## Visualize attention patterns

How attention patterns evolve from early to late processing

In [1]:
import circuitsvis as cv

In [2]:
from llava import LLaVA
llava = LLaVA()
llava.load_data(1)

/home/sofism/miniconda3/envs/llava-lens/lib/python3.10/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using `use_fast=True` but `torchvision` is not available. Falling back to the slow image processor.


Loaded pretrained model llama-7b-hf into HookedTransformer


Resolving data files:   0%|          | 0/200 [00:00<?, ?it/s]

In [3]:
layers_to_visualize = [1, 6, 11, 16, 21, 26, 31]

In [4]:
for image, question, image_name in llava.dataset:
    inputs_embeds, _, _, _, _ = llava.get_llm_input_embeddings(image, question)
    logits, cache = llava.hooked_llm.run_with_cache(inputs_embeds, start_at_layer=0, remove_batch_dim=True)
    for layer_to_visualize in layers_to_visualize:
        tokens_to_show = 40
        
        attention_pattern = cache["pattern", layer_to_visualize, "attn"]
        
        product = inputs_embeds @ llava.language_model.model.embed_tokens.weight.T.to(llava.device)  
        llama_str_tokens = llava.hooked_llm.to_str_tokens(product.argmax(dim=-1)[0])
        
        attention_html = cv.attention.attention_patterns(tokens=llama_str_tokens[-tokens_to_show:], 
                                                attention=attention_pattern[:, -tokens_to_show:, -tokens_to_show:])
        # display(attention_html)
        with open(llava.results_dir / f"{image_name}_{tokens_to_show}_{layer_to_visualize}_attention.html", 'w') as f:
            f.write(attention_html._repr_html_())